In [21]:
import numpy as np
import pandas as pd
import csv

In [24]:
"""
Ni–CeO₂(111) Site Quantification Script
---------------------------------------
Computes metal, interfacial, and support site counts for
Ni/CeO₂ catalysts based on geometric and structural parameters.

All calculations follow the user-provided formulas and
retain detailed inline comments for clarity.
"""
# -------------------------------------------------------------------
# Parameter storage structure
# -------------------------------------------------------------------
parameters = {
    'rm': [], 'rs': [], 'm': [], 's': [], 'n_ni': [],
    'Nem_net': [], 'NBm_net': [], 'NIm_net': [], 'NIs_net': [],
    'NNIs_net': [], 'D': [], 'Zmm': [], 's2m': [], 's2mi': [],
    'fa': [], 'n_particles_Ni': []
}


# -------------------------------------------------------------------
# Core computation
# -------------------------------------------------------------------
def calculate_sites(cat_wt: float, m_load: float, rm: float, ssa: float, rsm: float,
                    parameters: dict) -> dict:
    """
    Calculate Ni metal, interfacial, and support site properties.

    Parameters
    ----------
    cat_wt : float
        Total catalyst weight (in grams)
    m_load : float
        Total metal loading (in grams)
    rm : float
        Metal nanoparticle radius (in nm)
    ssa : float
        Specific surface area of the catalyst (in m²/g)
    rsm : float
        Interfacial radius (doughnut radius) in nm
    parameters : dict
        Dictionary to store all computed site data

    Returns
    -------
    dict
        Updated parameters dictionary with computed results
    """

    #Ni metal site quantification
    #metal [=] Ni (used interchangeably in the code)
    mass_ni = m_load * cat_wt  # total mass of Ni, m_Ni
    moles_ni = mass_ni / 58.693  # Total moles of Ni, n_Ni
    na = 6.022e23
    m = moles_ni * na  # Total Ni sites, N_Ni
    vol_ni = (2 / 3) * np.pi * (rm ** 3)  # Volume occupied by 1 Ni particle, V_p
    r_fcc = 0.3506 / (2 * (2 ** 0.5))  # Radius occupied by 1 Ni atom in the fcc bulk structure, r_fcc
    vol_1ni_fcc = (4 / 3) * np.pi * (r_fcc ** 3)  # Volume occupied by 1 fcc Ni atom, V_a
    n_ni = vol_ni / vol_1ni_fcc  # Total Ni atoms in 1 Ni particle, N_Ni,p
    CSA = 2 * np.pi * (rm ** 2)  # Curved Surface Area of 1 Ni particle, CAS_Ni,p
    vol_shell = CSA * (2 * r_fcc)  # Volume of the outermost shells of the hemispherical Ni particle, V_shell
    Nem = vol_shell / vol_1ni_fcc  # Total surface Ni atoms in 1 Ni particle, NS_Ni,p
    NBm = n_ni - Nem  # Total bulk Ni sites in 1 Ni particle
    D = Nem / n_ni  # Dispersion of Ni
    NIm = (np.pi * 2 * rm) / (2 * r_fcc)  # Total Ni interfacial NI sites in 1 Ni particle, NI_Ni,p
    Zmm = ((NIm * 4) + (Nem - NIm) * 6) / Nem  # Number of nearest Ni sites to a Ni site (coordination number)
    n_particles_Ni = m / n_ni  # Total Ni particles, N_p
    Nem_net = Nem * n_particles_Ni  # Total surface metal sites, NS_Ni,tot
    NBm_net = NBm * n_particles_Ni  # Total bulk metal sites
    NIm_net = NIm * n_particles_Ni  # Total interfacial metal sites, NI_Ni,tot
    TSA_Ni_projected = n_particles_Ni * np.pi * ((rm * 1e-9) ** 2)  # Total projected surface area occupied by Ni, A_m

    #CeO2(111) support site quantification
    # support [=] CeO2(111)
    TSA_ceo2 = ssa - TSA_Ni_projected  # Total projected surface area occupied by CeO2(111), A_s
    Zss = 3  # Number of nearest support Ov sites to an Ov site (coordination number)
    areal_density_o_sites = 15.45  # Areal density of lattice Ov on CeO2 (111)
    s = (areal_density_o_sites / 1e-18) * ssa  # Total Ov sites available in A_s, NS_s
    rs = rsm + rm  # Radius of the support, used to calculate interfacial sites
    NIs = np.pi * ((rs ** 2) - (rm ** 2)) * areal_density_o_sites  # Number of interfacial support sites around a single Ni nanoparticle, NI_s,p
    NIs_net = NIs * n_particles_Ni  # Total number of interfacial sites, NI_s
    NNIs_net = s - NIs_net  # Total number of non-interfacial sites, NNI_s
    s2m = s / Nem_net  # Ratio of total expose support to metal sites
    s2mi = NIs_net / NIm_net  # Ratio of total expose support to metal sites
    fa = TSA_Ni_projected / TSA_ceo2  # Fraction of area exposed by metal versus support

    # Store calculated results
    parameters['rm'].append(rm)
    parameters['rs'].append(rs)
    parameters['m'].append(m)
    parameters['s'].append(s)
    parameters['n_ni'].append(n_ni)
    parameters['Nem_net'].append(Nem_net)
    parameters['NBm_net'].append(NBm_net)
    parameters['NIm_net'].append(NIm_net)
    parameters['NIs_net'].append(NIs_net / NNIs_net)
    parameters['NNIs_net'].append(NNIs_net)
    parameters['D'].append(D * 100)
    parameters['Zmm'].append(Zmm)
    parameters['s2m'].append(s2m)
    parameters['s2mi'].append(s2mi)
    parameters['fa'].append(fa)
    parameters['n_particles_Ni'].append(n_particles_Ni)

    return parameters


# -------------------------------------------------------------------
# Execution block
# -------------------------------------------------------------------
def main() -> None:
    """Run calculations for a range of nanoparticle sizes and export results."""
    for x in np.linspace(4, 5, 5):
        cat_wt = 1      # what is total catalyst weight (in grams)?
        m_load = 0.04   # what is the total metal loading (in grams)?
        rm = x           # What is the metal nanoparticle radius (in nm)?
        ssa = 70        # Specific surface area of the catalyst (in m2/g)?
        rsm = 0.38      # Interfacial radius (doughnut radius) in nm?
        calculate_sites(cat_wt, m_load, rm, ssa, rsm, parameters)
        print(f"Finished radius: {rm}")

    # Save computed data to CSV
    with open('site_quantified_parameters.csv', 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(parameters.keys())
        writer.writerows(zip(*parameters.values()))

    print("\n✅ Results saved as 'site_quantified_parameters.csv'")


# -------------------------------------------------------------------
# Script entry point
# -------------------------------------------------------------------
if __name__ == "__main__":
    main()


Finished radius: 4.0
Finished radius: 4.25
Finished radius: 4.5
Finished radius: 4.75
Finished radius: 5.0

✅ Results saved as 'site_quantified_parameters.csv'
